In [4]:
import pandas as pd
import uuid
from sklearn.impute import SimpleImputer, KNNImputer

data = pd.read_csv('./data/HNP_StatsData.csv')

In [5]:
countries = [
  'CAN',
  'USA',
  'MEX',
  'IND',
  'CHN',
  'SDN',
  'BGD',
  'BRA',
  'NER'
]

education_keys = [
'SE.ADT.LITR.ZS',
'SE.ADT.LITR.FE.ZS',
'SE.ADT.LITR.MA.ZS',
'SE.PRM.ENRR',
'SE.PRM.ENRR.FE',
'SE.PRM.ENRR.MA',
'SE.SEC.ENRR',
'SE.SEC.ENRR.FE',
'SE.SEC.ENRR.MA',
'SE.XPD.TOTL.GD.ZS',
'SE.TER.ENRR',


'SE.PRM.CMPT.ZS',
]

health_keys = [
  'SH.DTH.COMM.ZS',
  'SH.DTH.NCOM.ZS',
  'SH.XPD.CHEX.GD.ZS',
  'SH.MED.BEDS.ZS',
  'SH.IMM.MEAS',
  'SH.STA.OWGH.ME.ZS',
  'SH.STA.OWGH.ME.ZS',
  'SH.TBS.INCD',
  'SH.ANM.CHLD.ZS',
  'SH.IMM.POL3',
  'SH.IMM.IDPT',
  'SH.STA.DIAB.ZS',
  'SH.UHC.SRVS.CV.XD',
  'SH.MED.NUMW.P3'
]

quality_of_life_keys = [
  'SH.STA.BASS.ZS',
  'SH.STA.SMSS.ZS',
  'SH.STA.WASH.P5',
  'SH.H2O.BASW.ZS',
  'SH.H2O.SMDW.ZS',
  'SL.TLF.TOTL.IN',
  'SL.UEM.TOTL.MA.ZS',
  'SL.UEM.TOTL.FE.ZS',
  'SH.MMR.WAGE.ZS',
  'SI.POV.NAHC',
  'SH.STA.ODFC.ZS',
  'SL.TLF.TOTL.FE.ZS',
  'SH.STA.AIRP.P5',
  'SH.STA.BRTC.ZS ',
]

population_keys = [
'SP.POP.TOTL',
'SP.DYN.LE00.MA.IN',
'SP.DYN.LE00.FE.IN ',
'SP.POP.GROW ',
'SP.DYN.LE00.IN',
'SM.POP.NETM',
'SP.RUR.TOTL',
'SP.RUR.TOTL.ZG',
'SI.POV.RUHC',
'SP.URB.TOTL',
'SP.URB.GROW',
'SI.POV.URHC',
'SP.POP.TOTL.MA.ZS',
'SP.POP.TOTL.FE.ZS',
]

columns = [
  'Country Name',
  'Country Code',
  'Indicator Name',
  'Indicator Code',
  '2005',
  '2006',
  '2007',
  '2008',
  '2009',
  '2010',
  '2011',
  '2012',
  '2013',
  '2014',
  '2015',
  '2016',
  '2017',
  '2018',
  '2019',
  '2020'
]

In [6]:
def getData(columns, keys, countries, tableName):
  selected_data = pd.DataFrame()
  selected_data_imputed = pd.DataFrame()
  for country in countries:
    current_info = data.loc[data['Country Code'] == country]
    current_info = current_info.loc[data['Indicator Code'].isin(keys)]
    current_info = current_info[columns]
    cols = current_info.columns.tolist()
    # convert Year into its own column
    current_info = current_info.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name= "Year", value_name="Value")
    cols = current_info.columns.tolist()
    cols.insert(2, cols.pop(cols.index("Year")))
    current_info = current_info.reindex(columns= cols) #Reposition the Year column into index 2
    current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
    current_info = current_info.pivot_table('Value', ['Country Name', 'Country Code', 'Year'], 'Indicator Name').reset_index() #Pivot the table so that Each value in indicator Name is its own column


    cols = current_info.columns.tolist()
    numeric_info = current_info.iloc[:,3:]
    imputer = KNNImputer()
    imputer.fit(numeric_info)
    sample_incomplete_rows = numeric_info[numeric_info.isnull().any(axis=1)].head()
    input_x = imputer.transform(numeric_info)
    imputed_data = pd.DataFrame(input_x, columns=numeric_info.columns, index=numeric_info.index)
    result = pd.concat([current_info.iloc[:,:3], imputed_data], axis=1, join="inner")

    # current_info.to_csv(path_or_buf=f'./seed_data/{country}_{tableName}finished.csv', columns=cols)
    selected_data_imputed = pd.concat((selected_data_imputed, result[cols]))
    selected_data = pd.concat((selected_data, current_info))
  selected_data_imputed.insert(0, 'key', [uuid.uuid4() for _ in range(len(selected_data_imputed.index))])
  selected_data.insert(0, 'key', [uuid.uuid4() for _ in range(len(selected_data.index))])
  cols.insert(0,'key')
  print(selected_data_imputed.head())
  selected_data_imputed.to_csv(path_or_buf=f'./seed_data/{tableName}_seed.csv', columns=cols, index=False)
  selected_data.to_csv(path_or_buf=f'./seed_data/{tableName}_Unimputed.csv', columns=cols, index=False)
tables = {
  'Education': education_keys,
  'Health' : health_keys,
  'Quality_of_life' : quality_of_life_keys,
  'Population': population_keys
  }
for key in tables:
  getData(columns, tables[key], countries, key)

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of D

Indicator Name                                   key Country Name  \
0               4fa7faa7-ead2-4d7f-867b-c20fa0b9ec55       Canada   
1               bde619c4-7c53-431f-ba76-32db091938d4       Canada   
2               4a90871b-cc33-4366-8263-b215be035d2d       Canada   
3               ddcbc686-8611-496e-b2ec-5cd50811a78b       Canada   
4               d2e321c7-1a54-4ff9-ad03-31b123433c3c       Canada   

Indicator Name Country Code  Year  \
0                       CAN  2005   
1                       CAN  2006   
2                       CAN  2007   
3                       CAN  2008   
4                       CAN  2009   

Indicator Name  Literacy rate, adult female (% of females ages 15 and above)  \
0                                                            99.0              
1                                                            99.0              
2                                                            99.0              
3                                         

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of D

Indicator Name                                   key Country Name  \
0               447f4c66-b269-4f55-aca5-73f56b8a45d9       Canada   
1               6c23cde0-36a7-481d-bc9c-a82f77cfff37       Canada   
2               ee973722-b506-4308-84b3-56cf27418d15       Canada   
3               45b988f6-d644-4485-894b-95814569a9e3       Canada   
4               d885ae79-9238-4ee8-81b3-8a5c65cb8a33       Canada   

Indicator Name Country Code  Year  \
0                       CAN  2005   
1                       CAN  2006   
2                       CAN  2007   
3                       CAN  2008   
4                       CAN  2009   

Indicator Name  Labor force, female (% of total labor force)  \
0                                                  46.419241   
1                                                  46.674059   
2                                                  46.855960   
3                                                  46.800794   
4                                         

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_69348/203642169.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
